In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset


pipe = pipeline("automatic-speech-recognition", model="openai/whisper-tiny")

C:\Users\arceu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'path': 'C:/Users/arceu/.cache/huggingface/datasets/downloads/extracted/b61d72a45049b10da968054078776a9138193cba9aa5ab93b61b530a9edcb627/dev_clean/1272/128104\\1272-128104-0000.flac',
 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
        0.0010376 ]),
 'sampling_rate': 16000}